# Lab 2: pH and Properties of Buffers

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Enzyme activity

In [2]:
# Read the csv
EA = pd.read_csv('/Users/kev/Documents/Python/BCMB-301A--Lab Techniques/Data/Lab 2/pH on Enzyme.csv', 
                 header=2)

EA = EA.set_index('Time (s)')

In [3]:
# Fine the derivative 
EA_d = EA.diff()
EA_d = EA_d.rename(index=str, columns={"3":"d3", "5":"d5", "7.5":"d7.5", "9":"d9", "10":"d10"})

In [4]:
# Concat the data with the derivative (buggy)
EA_conc = pd.concat([EA, EA_d], axis=1)

In [5]:
EA

,3,5,7.5,9,10
Time (s),,,,,
0,0.000,0.000,0.000,0.000,0.000
10,0.001,-0.096,-0.166,0.024,0.010
20,0.001,-0.155,-0.142,0.036,0.035
30,0.003,-0.146,-0.115,0.047,0.048
40,0.001,0.014,-0.090,0.060,0.068
50,0.000,0.087,-0.063,0.074,0.088
60,0.002,0.102,-0.037,0.088,0.105
70,0.001,0.104,-0.012,0.102,0.123
80,0.000,0.104,0.015,0.115,0.146


In [6]:
EA_d

,d3,d5,d7.5,d9,d10
Time (s),,,,,
0,NaN,NaN,NaN,NaN,NaN
10,0.001,-0.096,-0.166,0.024,0.010
20,0.000,-0.059,0.024,0.012,0.025
30,0.002,0.009,0.027,0.011,0.013
40,-0.002,0.160,0.025,0.013,0.020
50,-0.001,0.073,0.027,0.014,0.020
60,0.002,0.015,0.026,0.014,0.017
70,-0.001,0.002,0.025,0.014,0.018
80,-0.001,0.000,0.027,0.013,0.023


In [7]:
# Average enzyme activity 
EA_d_ave = EA_d.iloc[2:,:].mean(axis=0)
EA_d_ave

d3     -0.000091
d5      0.020818
d7.5    0.026364
d9      0.013636
d10     0.018727
dtype: float64

In [8]:
# Relative enzyme activity
EA_d_ave_percent = EA_d_ave / np.max(EA_d_ave) * 100
EA_d_ave_percent

d3       -0.344828
d5       78.965517
d7.5    100.000000
d9       51.724138
d10      71.034483
dtype: float64

In [9]:
plt.figure(figsize=(6, 4))
# EA_d_ave_percent.plot(kind='line')
# EA_d_ave_percent.plot(kind='scatter')
plt.plot(list(EA), EA_d_ave_percent)
plt.scatter(list(EA), EA_d_ave_percent)
plt.title("Percent enzyme activity vs pH")
plt.xlabel('pH')
plt.ylabel('Relative enzyme activity (%)')

Text(0, 0.5, 'Relative enzyme activity (%)')

<Figure size 432x288 with 1 Axes>

## Titration curve: Cubic spline fit

We need to have 6 curves: $S_0, S_1, S_2, S_3, S_4, S_5$

> $S_0$ connects point $x_0$ to $x_1$

> $S_1$ connects point $x_1$ to $x_2$

> $S_2$ connects point $x_2$ to $x_3$ 

> $S_3$ connects point $x_3$ to $x_4$

> $S_4$ connects point $x_4$ to $x_5$ 

> $S_5$ connects point $x_5$ to $x_6$

Values for the points (these correspond to the $pK_a$): 

> $x_0 = (-1.5, -0.29)$ 

> $x_1 = (-1, 2.35)$

> $x_2 = (-0.5, 4.4025)$

> $x_3 = (0, 7.63)$ 

> $x_4 = (0.5, 9.075)$

> $x_5 = (1, 10.52)$ 

> $x_6 = (1.5, 11.995)$

Equations for these points:

$$\left\{ \begin{array}{ll}
    S_0(x_0) = a_0 + b_0 x_0 + c_0 x_0^2 + d_0 x_0^3 \\
    S_0(x_1) = a_0 + b_0 x_1 + c_0 x_1^2 + d_0 x_1^3 \\
    S_1(x_1) = a_1 + b_1 x_1 + c_1 x_1^2 + d_1 x_1^3 \\
    S_1(x_2) = a_1 + b_1 x_2 + c_1 x_2^2 + d_1 x_2^3 \\
    S_2(x_2) = a_2 + b_2 x_2 + c_2 x_2^2 + d_2 x_2^3 \\
    S_2(x_3) = a_2 + b_2 x_3 + c_2 x_3^2 + d_2 x_3^3 \\
    S_3(x_3) = a_3 + b_3 x_3 + c_3 x_3^2 + d_3 x_3^3 \\
    S_3(x_4) = a_3 + b_3 x_4 + c_3 x_4^2 + d_3 x_4^3 \\
    S_4(x_4) = a_4 + b_4 x_4 + c_4 x_4^2 + d_4 x_4^3 \\
    S_4(x_5) = a_4 + b_4 x_5 + c_4 x_5^2 + d_4 x_5^3 \\
    S_5(x_5) = a_5 + b_5 x_5 + c_5 x_5^2 + d_5 x_5^3 \\
    S_5(x_6) = a_6 + b_5 x_6 + c_5 x_6^2 + d_5 x_6^3 
    \end{array} \right.$$    

We also want the derivatives at the connecting points to be zero: 

$$\left\{ \begin{array}{ll}
    S'_0(x_1) = b_0 + 2 c_0 x_1 + 3 d_0 x_1^2 = b_1 + 2 c_1 x_1 + 3 d_1 x_1^2 = S'_1(x_1) = 0\\
    S'_1(x_2) = b_1 + 2 c_1 x_2 + 3 d_1 x_2^2 = b_2 + 2 c_2 x_2 + 3 d_2 x_2^2 = S'_2(x_2) \\
    S'_2(x_3) = b_2 + 2 c_2 x_3 + 3 d_2 x_3^2 = b_3 + 2 c_3 x_3 + 3 d_3 x_3^2 = S'_3(x_3) = 0\\
    S'_3(x_4) = b_3 + 2 c_3 x_4 + 3 d_3 x_4^2 = b_4 + 2 c_4 x_4 + 3 d_4 x_4^2 = S'_4(x_4) \\
    S'_4(x_5) = b_4 + 2 c_4 x_5 + 3 d_4 x_5^2 = b_5 + 2 c_5 x_5 + 3 d_5 x_5^2 = S'_5(x_5) = 0\\
    \end{array} \right.$$   

For a natural spline, we want the second order derivatives to be continuous and to be zero at the ends: 

$$\left\{ \begin{array}{ll}
    S''_0(x_1) = 2 c_0 + 6 d_0 x_1 = 2 c_1 + 6 d_1 x_1 = S''_1(x_1)\\
    S''_1(x_2) = 2 c_1 + 6 d_1 x_2 = 2 c_2 + 6 d_2 x_2 = S''_2(x_2)\\
    S''_2(x_3) = 2 c_2 + 6 d_2 x_3 = 2 c_3 + 6 d_3 x_3 = S''_3(x_3)\\
    S''_3(x_4) = 2 c_3 + 6 d_3 x_4 = 2 c_4 + 6 d_4 x_4 = S''_4(x_4)\\
    S''_4(x_5) = 2 c_4 + 6 d_4 x_5 = 2 c_5 + 6 d_5 x_5 = S''_5(x_5)\\
    S_0''(x_0) = 2 c_0 + 6 d_0 x_0 = 2 c_5 + 6 d_5 x_6 = S_3''(x_6) = 0
    \end{array} \right.$$ 

Putting it all together:

$$\left\{ \begin{array}{ll}
    S_0(-1.5) = a_0 + b_0 x_0 + c_0 x_0^2 + d_0 x_0^3 \\
    S_0(-1) = a_0 - b_0 + c_0 - d_0 \\
    S_1(-1) = a_1 - b_1 + c_1 - d_1 \\
    S_1(0.5) = a_1 + b_1 x_2 + c_1 x_2^2 + d_1 x_2^3 \\
    S_2(0.5) = a_2 + b_2 x_2 + c_2 x_2^2 + d_2 x_2^3 \\
    S_2(0) = a_2 \\
    S_3(0) = a_3 \\
    S_3(0.5) = a_3 + b_3 x_4 + c_3 x_4^2 + d_3 x_4^3 \\
    S_4(0.5) = a_4 + b_4 x_4 + c_4 x_4^2 + d_4 x_4^3 \\
    S_4(1) = a_4 + b_4 + c_4 + d_4 \\
    S_5(1) = a_5 + b_5 + c_5 + d_5 \\
    S_5(1.5) = a_6 + b_5 x_6 + c_5 x_6^2 + d_5 x_6^3 \\
    S'_0(-1) = b_0 - 2 c_0 + 3 d_0 = b_1 - 2 c_1 + 3 d_1 = S'_1(-1) = 0\\
    S'_1(-0.5) = b_1 + 2 c_1 x_2 + 3 d_1 x_2^2 = b_2 + 2 c_2 x_2 + 3 d_2 x_2^2 = S'_2(-0.5) \\
    S'_2(0) = b_2 = b_3 = S'_3(0) = 0\\
    S'_3(0.5) = b_3 + 2 c_3 x_4 + 3 d_3 x_4^2 = b_4 + 2 c_4 x_4 + 3 d_4 x_4^2 = S'_4(0.5) \\
    S'_4(1) = b_4 + 2 c_4 + 3 d_4 = b_5 + 2 c_5 + 3 d_5 = S'_5(1) = 0\\
    S''_0(-1) = 2 c_0 - 6 d_0 = 2 c_1 - 6 d_1 = S''_1(-1)\\
    S''_1(-0.5) = 2 c_1 + 6 d_1 x_2 = 2 c_2 + 6 d_2 x_2 = S''_2(-0.5)\\
    S''_2(0) = 2 c_2  = 2 c_3 = S''_3(0)\\
    S''_3(0.5) = 2 c_3 + 6 d_3 x_4 = 2 c_4 + 6 d_4 x_4 = S''_4(0.5)\\
    S''_4(1) = 2 c_4 + 6 d_4 = 2 c_5 + 6 d_5 = S''_5(1)\\
    S_0''(-1.5) = 2 c_0 + 6 d_0 x_0 = 2 c_5 + 6 d_5 x_6 = S_3''(1.5) = 0
    \end{array} \right.$$ 

$$\left\{ \begin{array}{ll}
    S_0(-1.5) = a_0 + b_0 x_0 + c_0 x_0^2 + d_0 x_0^3 \\
    S_0(-1) = a_0 - b_0 + c_0 - d_0 \\
    S_1(-1) = a_1 - b_1 + c_1 - d_1 \\
    S_1(0.5) = a_1 + b_1 x_2 + c_1 x_2^2 + d_1 x_2^3 \\
    S_2(0.5) = a_2 + c_2 x_2^2 + d_2 x_2^3 \\
    S_2(0) = a_2 \\
    S_3(0) = a_3 \\
    S_3(0.5) = a_3 + c_2 x_4^2 + d_3 x_4^3 \\
    S_4(0.5) = a_4 + b_4 x_4 + c_4 x_4^2 + d_4 x_4^3 \\
    S_4(1) = a_4 + b_4 + c_4 + d_4 \\
    S_5(1) = a_5 + b_5 + c_5 + d_5 \\
    S_5(1.5) = a_6 + b_5 x_6 + c_5 x_6^2 + d_5 x_6^3 \\
    S'_0(-1) = b_0 - 2 c_0 + 3 d_0 = b_1 - 2 c_1 + 3 d_1 = S'_1(-1) = 0\\
    S'_1(-0.5) = b_1 + 2 c_1 x_2 + 3 d_1 x_2^2 = 2 c_2 x_2 + 3 d_2 x_2^2 = S'_2(-0.5) \\
    S'_3(0.5) = 2 c_2 x_4 + 3 d_3 x_4^2 = b_4 + 2 c_4 x_4 + 3 d_4 x_4^2 = S'_4(0.5) \\
    S'_4(1) = b_4 + 2 c_4 + 3 d_4 = b_5 + 2 c_5 + 3 d_5 = S'_5(1) = 0\\
    S''_0(-1) = 2 c_0 - 6 d_0 = 2 c_1 - 6 d_1 = S''_1(-1)\\
    S''_1(-0.5) = 2 c_1 + 6 d_1 x_2 = 2 c_2 + 6 d_2 x_2 = S''_2(-0.5)\\
    S''_3(0.5) = c_2 + 3 d_3 x_4 = c_4 + 3 d_4 x_4 = S''_4(0.5)\\
    S''_4(1) = c_4 + 3 d_4 = c_5 + 3 d_5 = S''_5(1)\\
    S_0''(-1.5) = c_0 + 3 d_0 x_0 = c_5 + 3 d_5 x_6 = S_3''(1.5) = 0
    \end{array} \right.$$ 

In [10]:
x0 = -1.5
x1 = -1
x2 = -0.5
x3 = 0
x4 = 0.5
x5 = 1
x6 = 1.5

[y0, y1, y2, y3, y4, y5, y6] = [-0.29, 2.35, 4.4025, 7.63, 9.075, 10.52, 11.995]

C = np.array([
    [1, x0,  x0**2,  x0**3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [1, -1,  1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  1, -1,  1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  1, x2, x2**2, x2**3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  1,  0, x2**2, x2**3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0, x4**2,  0,  1,  0,  0, x4**3,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, x4, x4**2, x4**3,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  x6,  x6**2,  x6**3],
    [0,  0,  0,  0,  0,  1, -2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  1, -2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  1,  2*x2,  3*x2**2,  0,  0, -2*x2, -3*x2**2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 2*x4, 0,  0,  0,  0, 3*x4**2, -1, 2*x4, 3*x4**2,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,  3,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -2, -3,  0,  0,  0,  0],
    [0,  0,  1, -3,  0,  0, -1,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  1, 3*x2,  0,  0, -1,-3*x2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0, 3*x4,  0,  0, -1, -3*x4,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, -3,  0,  0, -1, -3,],
    [0,  0,  1, 3*x0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 3*x6]])

# Create an array of vector "b"
vec_b = np.array([
    [y0],
    [y1],
    [y1],
    [y2], 
    [y2], 
    [y3],  
    [y3], 
    [y4],  
    [y4], 
    [y5],
    [y5], 
    [y6], 
    [0], 
    [0], 
    [0], 
    [0],
    [0], 
    [0], 
    [0], 
    [0], 
    [0], 
    [0], 
    [0], 
    [0]])

Now, let's put this into a matrix:

$$C \vec{a} = \vec{b}$$

$$\text{Where:} \ 
\left\{ \begin{array}{ll}
            C \ \text{is the matrix containing the coefficient of "a's", "b's", "c's", and "d's"}\\
            \vec{a} \ \text{is a vector containing the "a's", "b's", "c's", and "d's"}\\
            \vec{b} \ \text{is a vector containing the number on the right side of the equations}
            \end{array} \right.$$ 


The coefficients can be calculated by: $$\vec{a} = C^{-1} \vec{b}$$

In [11]:
# Find the inverse of "C"
C_inv = np.linalg.inv(C)

# Take the product of "C" and "b"
vec_a = C_inv @ vec_b
print(r"a = ", vec_a)

LinAlgError: Singular matrix

This is our coefficients for the cubic spline equations.

In [ ]:
x_space0 = np.arange(-2, -1, 0.01)
x_space1 = np.arange(-1, 0, 0.01)
x_space2 = np.arange(0, 1, 0.01)
x_space3 = np.arange(1, 2, 0.01)

def S(A=vec_a, x0=x_space0, x1=x_space1, x2=x_space2, x3=x_space3):
    s0 = A[0] + A[1]*x0 + A[2]*x0**2 + A[3]*x0**3
    s1 = A[4] + A[5]*x1 + A[6]*x1**2 + A[7]*x1**3
    s2 = A[8] + A[9]*x2 + A[10]*x2**2 + A[11]*x2**3
    s3 = A[12] + A[13]*x3 + A[14]*x3**2 + A[15]*x3**3
    return s0, s1, s2, s3

In [ ]:
# Plot it out:
# ============
S0, S1, S2, S3 = S()

fig, ax2 = plt.subplots(1, 1, figsize=(10, 6))

ax2.scatter(P[:,0], P[:,1], c='black', label="Data points", zorder=5)
ax2.plot(x_space0, S0, label=r"$S_0(x) = {0:.2f} + {1:.2f} x + {2} x^2 + {3} x^3$"
         .format(vec_a[0,0], vec_a[1,0], vec_a[2,0], vec_a[3,0]), linewidth=2)
ax2.plot(x_space1, S1, label=r"$S_1(x) = {0} + {1} x + {2:.2f} x^2 + {3:.2f} x^3$"
         .format(vec_a[4,0], vec_a[5,0], 0, vec_a[7,0]), linewidth=2)
ax2.plot(x_space2, S2, label=r"$S_2(x) = {0} + {1} x + {2:.2f} x^2 + {3:.2f} x^3$"
         .format(vec_a[8,0], vec_a[9,0], 0, vec_a[11,0]), linewidth=2)
ax2.plot(x_space3, S3, label=r"$S_3(x) = {0:.2f} + {1:.2f} x + {2:.2f} x^2 + {3:.2f} x^3$"
         .format(vec_a[12,0], vec_a[13,0], vec_a[14,0], vec_a[15,0]), linewidth=2)
ax2.plot(x_space, sin(pi*x_space / 2), label=r"$y=\sin \left(\frac{2 x}{\pi}\right)$", linewidth=4, zorder=0)

ax2.set_ylabel(r"$y$", rotation=0, fontsize=14)
ax2.set_xlabel(r"$x$", fontsize=14)
ax2.set_title(r"Cubic spline fit", fontsize=16)

plt.ylim(-1.75, 1.25)
plt.legend()
plt.show()

In [13]:
from scipy.interpolate import BPoly

In [27]:
Test = BPoly.from_derivatives([0, 1, 2], [[1], [2, 0], [3]])

In [30]:
Test_t = np.linspace(0, 2, 100)

In [31]:
plt.plot(Test_t, Test(Test_t))

<Figure size 432x288 with 1 Axes>